# Tomographic reconstruction from a rectangular array

In this exercise, you will go from setting up a model of tomographic measurement to reconstructing an attenuation distribution from noisy beam measurements.

The domain in which the attenuation distribution is reconstructed is rectangular, of given width and length. In the particular example that you will work with, this domain is square of length 1 meter, but length and width could have any value. You can assume that the lower-left corner of the reconstructed domain is the origin itself.

You'll work with algebraic reconstruction, so the first thing that you probably want to do is to discretize the domain by representing it as a weighted sum of basis functions. To make things simple, we suggest you choose basis functions with the shape of a disc whose radius $r$ is related to interpixel distance $d$ by $r = \tfrac{d}{\sqrt{2}}$, as illustrated in the figure below.

<img src="grid_round_pixels.png">

We also suggest to model beams as infinitely thin lines, such that the inner product between the k-th pixel and i-th beam, $\langle \phi_k, h_i \rangle$, equals the length of the beam's segment that lies inside that pixel, as illustrated in the following figure:

<img src="beam_and_round_pixel.png">

In the first part of the exercise, you will construct the measurement matrix $A$. In order to help you achieve this, we suggested to implement a few functions that can be used to that end; however, if you prefer to do it differently, you're welcome to take your own route.

Beams are lines defined by their endpoints, given in two matrices: **beam_start** and **beam_end**. Each is of size $n_{\it beams}\times 2$, where $n_{\it beams}$ is the number of beams. The i-th row of **beam_start** contains the $(x,y)$ coordinates of the start of beam $i$, while the i-th row of **beam_end** contains the $(x,y)$ coordinates of the end of beam $i$. Those two matrices should be enough for you to determine the line parametrization, given by $(\theta_i,t_i)$, of every beam. If you forgot, $\theta\in[0,\pi)$ is the angle of the line normal, while $t$ is the signed distance of the line to the origin, along the direction defined by $\theta$.

When you have computed the line parameterization for all the beams, you can compute the inner products $\langle \phi_k, h_i \rangle$ mentioned above by finding the length of the intersection between beams and round pixels. With those, you have everything you need to populate the measurement matrix $A$.

Once you have the measurement matrix $A$, you should be able to reconstruct the pixel values. The vector with measurements, which we denoted by $b$ in the slides, is given in the variable called **measurement**.

The algorithms for algebraic reconstruction were described in the lecture and have been shown in the application session. To simplify your life, we provide a separate file **helpers.py** with some of the functions that have been given to you in the solution to the application session, but you're welcome to use the one that you were given earlier this week.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
import scipy.io
import tqdm

%matplotlib inline

In [ ]:
# Load the relevant data from the given .mat file.
data = scipy.io.loadmat('hw2_ex3.mat')

# Read the measurement vector and store it as an array b.
b = data['measurement'].squeeze()

# Beam start and beam end points.
beam_start = data['beam_start']
beam_end = data['beam_end']

# Length and width of the reconstruction domain (in meters, not in pixels).
length = data['length']
width = data['width']

# Arrays containing positions of all emitters and all detectors
# NOTE: not all beams that can be measured are actually measured,
#       that's why the measured beams are defined with beam_start and beam_end
emitter_pos = data['emitter_pos']
detector_pos = data['detector_pos']

# Plot the emitters and detectors.
fig, ax = plt.subplots(1, 1, figsize = (10,10))
plt.scatter(emitter_pos[:,0], emitter_pos[:,1], label='emitters')
plt.scatter(detector_pos[:,0], detector_pos[:,1], label='detectors')
plt.grid()
plt.legend()
plt.title("Array of emitters and detectors")
plt.show()


## Generate the measurement matrix

In [ ]:
def compute_beam_line_params(beam_start, beam_end):
    """
    Compute the line parameters between points in beam_start and points in beam_end.
    
    NOTE: Make sure that the line normals are between 0 and \pi.
    
    Arguments:
        beam_start: (n_beams, 2) matrix with emitter positions
        beam_end: (n_beams, 2) matrix with detector positions
    
    Returns: 
        Matrix (n_beams, 2) of (unit-norm) line normals.
        Vector (n_beams,) of line signed-distances from the origin.
    """
    beam_angles = np.arctan2(beam_end[:,1]-beam_start[:,1], beam_end[:,0]-beam_start[:,0])
    # Rotate by \pi/2 to get the angle of a beam normal (rotation by -\pi/2 would also work).
    beam_normals = beam_angles + np.pi / 2
    # Make corrections to have angles in the range [0,\pi], by now they are in [-\pi/2,3\pi/2].
    beam_normals[beam_normals > np.pi] -= 2 * np.pi
    beam_normals[beam_normals < 0] += np.pi
    beam_vectors = np.stack((np.cos(beam_normals), np.sin(beam_normals)), axis=-1)
    # Distance from any point on a line is the same, so just compute using beam_start.
    beam_dist = np.sum(beam_start * beam_vectors, axis=-1)

    return beam_vectors, beam_dist


def beam_pixel_intersection_length(beam_normals, beam_dist, pix_pos, pix_r):
    """
    Compute the lenght of the intersection between a thin beam and a round (disc) pixel.
    
    Arguments:
        beam_normals: (2,) vector (\cos(\theta), \sin(\theta)) normal to the beam
        beam_dist: signed distance of the beam line to the origin
        pix_pos: (2,) vector with (x,y) coordinates of the pixel
        pix_r: radius of the pixel disc
    
    Returns:
        Length of the beam segment inside the pixel disc.
    """
    dist = np.abs(np.sum(beam_normals * pix_pos, axis=-1) - beam_dist)
    # Clipping distance to pix_r to simplify length computation.
    dist = np.minimum(dist, pix_r)
    intersection_length = 2 * np.sqrt(pix_r**2 - dist**2)
    
    return intersection_length


def construct_forward_matrix(beam_normals, beam_dist, pixel_pos, pixel_r):
    """
    Constructs the forward matrix from beam line parameters and pixel grid parameters.
    
    Arguments:
        beam_normals: (n_beams, 2) matrix with beam normal vectors (\cos(\phi_i), \sin(\phi_i) in every row
        beam_dist: (n_beams,) vector with beams' signed distances from the origin
        pixel_pos: (n_pixels, 2) matrix with pixel coordinates (x_i,y_i) in every row
        pixel_r: pixel radius
    
    Returns:
        Measurement matrix of size (n_beams, n_pixels) for the given beams
        and the given pixel grid.
    """
    # Here we're using a relatively ugly but probably more efficient way of computing coefficients
    # (a simple and more time-consuming way would use nested for loops).
    # We split the x and y coordinates of beam normals and pixel positions in MxN matrices,
    # which allow us to compute distances with simple matrix elemenet-wise products and sums.
    beam_x = np.kron(np.reshape(beam_normals[:,0], (-1,1)), np.ones((1, pixel_pos.shape[0])))
    beam_y = np.kron(np.reshape(beam_normals[:,1], (-1,1)), np.ones((1, pixel_pos.shape[0])))
    pixel_x = np.kron(np.reshape(pixel_pos[:,0], (1,-1)), np.ones((beam_normals.shape[0], 1)))
    pixel_y = np.kron(np.reshape(pixel_pos[:,1], (1,-1)), np.ones((beam_normals.shape[0], 1)))
    beam_dist_mat = np.kron(np.reshape(beam_dist, (-1,1)), np.ones((1, pixel_pos.shape[0])))
    # Now the distance computation becomes simple.
    # It could probably be even simpler and more elegant with multidimensional matrices,
    # but let's leave it at this.
    beam_pixel_dist = np.abs(beam_x * pixel_x + beam_y * pixel_y - beam_dist_mat)
    # Anything above pixel radius clipped to the radius which makes the coefficient zero.
    beam_pixel_dist = np.minimum(beam_pixel_dist, pixel_r)
    forward_matrix = 2 * np.sqrt(pixel_r**2 - beam_pixel_dist**2)
    
    return forward_matrix
    

# Here we proceed by computing beam_normals and beam_dist from beam_start and beam_end
beam_normals, beam_signed_dist = compute_beam_line_params(beam_start, beam_end)

# Then we generate the pixel grid. 
# Hint 1: the number of pixels could be equal to the number of measurements
# Hint 2: set pixel sizes according to the number of pixels and the size of the reconstructed domain.
n_pixels = len(b) 

# Pixel distances along x and y.
dx = np.sqrt(width * length / n_pixels)

# Pixel size (radius) along x and y (pixels modeled as discs).
pixel_r = dx / np.sqrt(2)

# Generate the pixel grid
pixel_grid_x = np.arange(dx/2, width, dx)
pixel_grid_y = np.arange(dx/2, length, dx)

nx = len(pixel_grid_x)
ny = len(pixel_grid_y)

pixel_x = np.kron(np.reshape(pixel_grid_x, (-1,1)), np.ones((pixel_grid_y.shape[0],1)))
pixel_y = np.kron(np.ones((pixel_grid_x.shape[0],1)), np.reshape(pixel_grid_y, (-1,1)))

pixel_pos = np.concatenate((pixel_x, pixel_y), axis=-1)

# Finally, we compute the forward matrix (i.e. the measurement matrix).
A = construct_forward_matrix(beam_normals, beam_signed_dist, pixel_pos, pixel_r)


## Reconstruction based on the measurements

Now it's time to try to reconstruct the image from measurements. You should use Kaczmarz algorithm that you can implement yourself (interface provided below), and the version you got in the application session earlier this week can be a good start if you don't like to write things from scratch.

1. Reconstruct the image using the simplest Kaczmarz algorithm with $2 n_{\it beams}$ iterations.
2. Reconstruct the image using a variation of randomized Kaczmarz algorithm with $2 n_{\it beams}$ iterations. There are several ways in which you can randomize the order in which projections are applied:
    * one variant of the randomized Kaczmarz algorithm projects one row at a time, where rows are not swept through sequentially, but rather at every iteration $k$, a row is selected randomly with probability proportional to its squared $l_2$-norm, $\|r_i\|_2^2$
    * another variant of the randomized Kaczmarz algorithm sweeps through the rows sequentially, as in the simple version of the algorithm, but rows are randomly permuted at the start.
3. Try to reconstruct the image using a modified randomized Kaczmarz algorithm with $2 n_{\it beams}$ iterations, where upon computing the projection at every iteration $k$, you apply the box constraint $P_{\cal{C}}(f^{(k)})$, where ${\cal{C}} = [0,1]^{n_{\it pixels}}$ and $n_{\it pixels}$ is the number of pixels.
4. Show the results of the three reconstructions.
5. Do you notice any difference in the quality of the reconstructions? Which algorithm gives the best results?

The images should be two-dimensional. If they appear mirrored or rotated, try to transform them such that you get a good orientation (if your reconstruction is good, you'll have an idea what correction you need).

In [ ]:
def kaczmarz(A, b, n_iter, limits=None, randomize=False, f_0=None):
    """
    Form image via Kaczmarz's algorithm.

    Parameters
    ----------
    A : :py:class:`~numpy.ndarray`
        (n_beams, n_pixels) measurement matrix.
    b : :py:class:`~numpy.ndarray`
        (n_beams,) vector with measurements.
    n_iter : int
        Number of iterations to perform.
    limits : :py:class:`~numpy.ndarray`
        (2,) pixel value constraints. 
        Each pixel of the output image must lie in [`limits[0]`, `limits[1]`].
        If `None`, then the range is not restricted.
    randomize: bool
        Apply a ranomization strategy when applying projections.
    f_0 : :py:class:`~numpy.ndarray`
        (n_pixels,) initial point of the optimization.
        If unspecified, the initial point is set to an all-zero vector.

    Returns
    -------
    f : :py:class:`~numpy.ndarray`
        (n_pixels,) vectorized image
    """
    n_beams, n_pixels = A.shape

    f = np.zeros((n_pixels,), dtype=float) if (f_0 is None) else f_0.copy()

    if randomize:
        norms_sq = np.sum(A * A, axis=-1)
        probs = norms_sq / sum(norms_sq)
        index = np.random.choice(n_beams, size=n_iter, p=probs)
    else:
        index = np.arange(n_iter) % n_beams
        
    for k in tqdm.tqdm(index):
        n, s = b[k], A[k,:]
        l = s @ s

        if ~np.isclose(l, 0):
            # `l` can be very small, in which case it is dangerous to do the rescale. 
            # We'll simply drop these degenerate basis vectors.
            scale = (n - s @ f) / l
            f += scale * s
        
        if limits:
            f = np.clip(f, limits[0], limits[1])

    return f

In [ ]:
im0 = kaczmarz(A, b, 2*A.shape[0], randomize=False, limits=None).reshape(ny, nx)
im1 = kaczmarz(A, b, 2*A.shape[0], randomize=True, limits=None).reshape(ny, nx)
im2 = kaczmarz(A, b, 2*A.shape[0], randomize=True, limits=[0,1]).reshape(ny, nx)

def plot_image(I, title=None, ax=None):
    """
    Plot a 2D mono-chromatic image.

    Parameters
    ----------
    I : :py:class:`~numpy.ndarray`
        (n_height, n_width) image.
    title : str
        Optional title to add to figure.
    ax : :py:class:`~matplotlib.axes.Axes`
        Optional axes on which to draw figure.
    """
    if ax is None:
        _, ax = plt.subplots()

    ax.imshow(I, cmap='bone')

    if title is not None:
        ax.set_title(title)

fig, (ax0, ax1, ax2) = plt.subplots(1,3, figsize = (15,10))

plot_image(im0, title='Kaczmarz', ax=ax0)
plot_image(im1, title='randomized Kaczmarz', ax=ax1)
plot_image(im2, title='randomized Kaczmarz with box constraints', ax=ax2)

## Observations

The reconstruction noise is not only due to the limitations of the reconstruction method, but also due to the noise that we added to the generated measurements.

We can observe that the randomization of projection order breaks the structure of the reconstruction artifacts and one can argue that it accelerates convergence.

One can however have little doubt about the effectiveness of the box constraint applied at every iteration, as it is clear that it improves the quality of the reconstructed image.

## Further instructions

You can supply your solution to this practical exercise by sending me (Mihailo) an email with your Jupyter notebook that contains the solutions, which consists of the code and reconstructed images with your observations written in textual form. You can also put your solution on Github or some other publicly accessible repository and send us a link.

To keep the amount of sent data to the minimum, to you don't need to include the .mat files that you got in the archive.

If you have any question, don't hesitate to ask me during or after a lecture, or just send me an email.